In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time
from riotwatcher import LolWatcher, ApiError
import os
from tqdm.notebook import tqdm
import glob

In [54]:
API_KEY = 'RGAPI-60f0dc46-d361-44f4-8e6c-5f2a7d9b5952'

## PART 5: MATCH PROCESSING

In [29]:
def get_match_participants(match_id):
    with open(f'match_data/{match_id}.txt') as json_file:
        match_detail = json.load(json_file)        
    
    game_duration = match_detail['gameDuration'] / 60
    
    participants = []
    for row in match_detail['participants']:
        participants_row = {}
        participants_row['gameDuration'] = game_duration
        participants_row['champion'] = row['championId']
        participants_row['teamId'] = row['teamId']
        participants_row['spell1'] = row['spell1Id']
        participants_row['spell2'] = row['spell2Id']
        participants_row['win'] = row['stats']['win']
        participants_row['kills'] = row['stats']['kills']
        participants_row['deaths'] = row['stats']['deaths']
        participants_row['assists'] = row['stats']['assists']
        participants_row['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
        participants_row['goldEarned'] = row['stats']['goldEarned']
        participants_row['champLevel'] = row['stats']['champLevel']
        participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
        participants_row['item0'] = row['stats']['item0']
        participants_row['item1'] = row['stats']['item1']
        participants_row['role'] = row['timeline']['role']
        participants_row['lane'] = row['timeline']['lane']
        participants.append(participants_row)

    df = pd.DataFrame(participants)
    return df

In [32]:
def get_player_performance(participants, role, lane, champion):
    summoner_role = role
    summoner_lane = lane
    summoner_champion = champion
    
    game_duration = participants['gameDuration'].values[0]

    player = participants.loc[participants['champion'] == summoner_champion]
    if len(player) != 1:
        print('Player identification error: ignoring results from match.')
        return
    
    # Capture direct matchup opponent information
    matchup_opponent = participants.loc[participants['role'] == summoner_role].loc[participants['lane'] == (('MIDDLE' or 'MID') if summoner_lane == ('MID' or 'MIDDLE') else summoner_lane)].loc[participants['champion'] != summoner_champion]
    if len(matchup_opponent) != 1:
        print('Matchup opponent identification error: ignoring results from match.')
        print('Unable to match role: {}:{}'.format(summoner_role, summoner_lane))
        return
    
    # Collect team information
    team = player['teamId'].values[0]

    team_kills = np.sum(participants.loc[participants['teamId'] == team]['kills'])
    team_deaths = np.sum(participants.loc[participants['teamId'] == team]['deaths'])
    team_gold = np.sum(participants.loc[participants['teamId'] == team]['goldEarned'])
    team_damage = np.sum(participants.loc[participants['teamId'] == team]['totalDamageDealtToChampions'])
    
    player_performance = pd.DataFrame()
    
    player_performance['MatchupGoldDiff'] = player['goldEarned'].values - matchup_opponent['goldEarned'].values
    player_performance['KillParticipation'] = np.round((player['kills'].values + player['assists'].values) / team_kills * 100, 2)
    player_performance['DeathParticipation'] = np.round(player['deaths'].values / team_deaths * 100, 2)
    player_performance['GoldShare'] = np.round(player['goldEarned'].values / team_gold * 100, 2)
    player_performance['DamageShare'] = np.round(player['totalDamageDealtToChampions'].values / team_damage * 100, 2)
    player_performance['DamagePerMin'] = np.round(player['totalDamageDealtToChampions'].values / game_duration, 2)
    player_performance['MinionsKilledPerMin'] = np.round(player['totalMinionsKilled'].values / game_duration, 2)
    player_performance['GameDuration'] = np.round(game_duration, 2)
    player_performance['Champion'] = player['champion'].values
    player_performance['role'] = summoner_role + summoner_lane
    player_performance['isWin'] = player['win'].values
    
    return player_performance

## PART 6: SUMMARIZE PLAYER

In [68]:
# Collect champion id key pairs and create a reference dictionary to translate key numbers

response = requests.get('http://ddragon.leagueoflegends.com/cdn/10.10.3216176/data/en_US/champion.json')

static_champ_list = response.json()

CHAMP_DICT = {}
for key in static_champ_list['data']:
    row = static_champ_list['data'][key]
    CHAMP_DICT[row['key']] = row['id']

In [79]:
def boil_history(match_history_df, player_name):
    player_summary = pd.DataFrame({
        'GamesPlayed': len(match_history_df),
        'WinRate': np.round(len(match_history_df.loc[match_history_df['isWin'] == True]) / len(match_history_df) * 100, 2),
        'WinGoldDiff': match_history_df.loc[match_history_df['isWin'] == True]['MatchupGoldDiff'].mean(),
        'WinKP': match_history_df.loc[match_history_df['isWin'] == True]['KillParticipation'].mean(),
        'WinDP': match_history_df.loc[match_history_df['isWin'] == True]['DeathParticipation'].mean(),
        'WinGoldShare': match_history_df.loc[match_history_df['isWin'] == True]['GoldShare'].mean(),
        'WinDmgShare': match_history_df.loc[match_history_df['isWin'] == True]['DamageShare'].mean(),
        'WinDmgRate': match_history_df.loc[match_history_df['isWin'] == True]['DamagePerMin'].mean(),
        'WinCSRate': match_history_df.loc[match_history_df['isWin'] == True]['MinionsKilledPerMin'].mean(),
        'WinTime': match_history_df.loc[match_history_df['isWin'] == True]['GameDuration'].mean(),
        'WinningestRole': match_history_df.loc[match_history_df['isWin'] == True]['role'].mode()[0],
        'WinningestChampion': CHAMP_DICT[str(match_history_df.loc[match_history_df['isWin'] == True]['Champion'].mode()[0])],

        'LoseGoldDiff': match_history_df.loc[match_history_df['isWin'] == False]['MatchupGoldDiff'].mean(),
        'LoseKP': match_history_df.loc[match_history_df['isWin'] == False]['KillParticipation'].mean(),
        'LoseDP': match_history_df.loc[match_history_df['isWin'] == False]['DeathParticipation'].mean(),
        'LoseGoldShare': match_history_df.loc[match_history_df['isWin'] == False]['GoldShare'].mean(),
        'LoseDmgShare': match_history_df.loc[match_history_df['isWin'] == False]['DamageShare'].mean(),
        'LoseDmgRate': match_history_df.loc[match_history_df['isWin'] == False]['DamagePerMin'].mean(),
        'LoseCSRate': match_history_df.loc[match_history_df['isWin'] == False]['MinionsKilledPerMin'].mean(),
        'LoseTime': match_history_df.loc[match_history_df['isWin'] == False]['GameDuration'].mean(),
        'LosingestRole': match_history_df.loc[match_history_df['isWin'] == False]['role'].mode()[0],
        'LosingestChampion': CHAMP_DICT[str(match_history_df.loc[match_history_df['isWin'] == False]['Champion'].mode()[0])],
    }, index=[player_name])
    
    return player_summary

In [72]:
def boil_player(player_name):
    with open(f'matchlist_data/{player_name}.txt') as json_file:
        matches = json.load(json_file)['matches']
    
    matches_df = pd.DataFrame()
    
    for match in matches:
        if match['queue'] == 420:
            print("Processing match {}".format(match['gameId']))
            match_id = match['gameId']
            role = match['role']
            lane = match['lane']
            champion = match['champion']
            
            try:
                participants = get_match_participants(match_id)
            except FileNotFoundError:
                continue
            
            player_performance = get_player_performance(participants, role, lane, champion)
            
            matches_df = pd.concat([matches_df, player_performance])
            
    if len(matches_df) <= 10:
        print(f'Skipping {player_name}: has not played enough solo queue.')
        return
        
    boiled = boil_history(matches_df, player_name)
    
    with open('boiled_players.csv', 'a') as f:
        boiled.to_csv(f, header=f.tell()==0)
    
    print('DONE')

In [42]:
boil_player('heungheng')

Processing match 3428189439
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:MID
Processing match 3428174116
Processing match 3423529251
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3416364192
Processing match 3412614962
Processing match 3412623679
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3412621003
Processing match 3412519069
Processing match 3412122234
Processing match 3412004441
Processing match 3411570748
Processing match 3387009798
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3382518744


## PART 7: SUMMARIZE EVERY PLAYER IN LEAGUE

In [48]:
def boil_league(league_name):
    player_list = [os.path.basename(x).split('.')[0] for x in glob.glob(f'summoner_data/{league_name}/*.txt')]
    print(f'Boiling {len(player_list)} players from {league_name}.')
    
    for player in tqdm(player_list):
        print(f'Boiling {player}.')
        boil_player(player)
    
    print('DONE')

In [51]:
boil_league('GOLD_Darius\'s Runemasters')

Boiling 192 players from GOLD_Darius's Runemasters.


Boiling heungheng.
Processing match 3428189439
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:MID
Processing match 3428174116
Processing match 3423529251
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3416364192
Processing match 3412614962
Processing match 3412623679
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3412621003
Processing match 3412519069
Processing match 3412122234
Processing match 3412004441
Processing match 3411570748
Processing match 3387009798
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3382518744
Skipping heungheng: has not played enough solo queue.
Boiling Astrid.
Skipping Astrid: has not played enough solo queue.
Boiling DrTy LUXury.
Processing match 3415974876
Matchup opponent identific

Processing match 3425407007
Processing match 3425271825
Processing match 3414324779
Processing match 3414205608
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3411832891
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3411602715
Processing match 3410576973
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3408499388
Processing match 3408484654
Processing match 3408460840
Processing match 3405712401
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3405670053
Processing match 3403765162
Processing match 3403698821
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:MID
Processing match 3403642988
Processing match 3403066942
Processing match 3402971992
Matchup opponent identif

Processing match 3281319398
Processing match 3280545822
Processing match 3280551645
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3280483745
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3280439006
Processing match 3279432253
Processing match 3279390771
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3279325448
Processing match 3279350018
Processing match 3278297985
Processing match 3278293060
Processing match 3278227799
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3277578128
Processing match 3277125515
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3277131705
Processing match 3277100198
Processing match 3277048053
Processing match 327683

Processing match 3277618470
Processing match 3277631140
Processing match 3277265300
Processing match 3277269884
Processing match 3277175704
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3276777126
Processing match 3276751577
Processing match 3276673223
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3276597240
Processing match 3276600258
Processing match 3276544213
Processing match 3275529287
Processing match 3275533273
Processing match 3275445923
Processing match 3275379958
Processing match 3275323258
Processing match 3275229308
Processing match 3275243608
Processing match 3275166641
Processing match 3275141487
Processing match 3275115760
Processing match 3274948639
Processing match 3274916466
Processing match 3274904149
Processing match 3274765043
Processing match 3274730592
Processing match 3274716722
Matchup opponent identification err

...

Processing match 3322422724
Processing match 3322353506
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3322320931
Processing match 3322021203
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3322006692
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3321992848
Processing match 3321527247
Processing match 3321494824
Processing match 3321500553
Processing match 3321148699
Processing match 3321143982
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3321008959
Processing match 3321000807
Processing match 3320568037
Processing match 3320563236
Processing match 3320161483
Processing match 3320051019
Processing match 3320014070
Processing match 3319938348
Matchup opponent identification error: ignoring 

Processing match 3315663458
Processing match 3315651538
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3314394027
Processing match 3314381254
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3314315824
Processing match 3314243666
Processing match 3313387763
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3313403586
Processing match 3301965903
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:MID
Processing match 3301983793
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3301982057
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3301390173
Matchup opponent identificati

Processing match 3399499880
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:MID
Processing match 3399408608
Processing match 3399284234
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3398969919
Processing match 3397739000
Processing match 3396923361
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3383202183
Processing match 3383188414
Processing match 3380216855
Processing match 3380214390
Processing match 3380221816
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:TOP
Processing match 3380128588
Processing match 3380145644
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3380171353
Processing match 3378569336
Processing match 3378527518
Processing match 3378564635
Processing 

Processing match 3386591423
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3385234198
Processing match 3380469688
Processing match 3380348796
Processing match 3380333660
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3378889329
Processing match 3378884356
Processing match 3377666955
Processing match 3377661308
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3377645220
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3377513072
Processing match 3377381700
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3377327832
Processing match 3377354492
Processing match 3375738798
Matchup opponent identification error: ignoring results from match.
Unable to m

Processing match 3401956605
Processing match 3401991029
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3401922899
Processing match 3401855280
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3401776811
Processing match 3401709724
Processing match 3401052561
Processing match 3401019196
Processing match 3400994152
Processing match 3401001012
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3400918548
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3400902099
Processing match 3400874603
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3400777933
Processing match 3400781561
Processing match 3400717199
Matchup opponent identifi

In [80]:
boil_league('SILVER_Jax\'s Dragons')

Boiling 179 players from SILVER_Jax's Dragons.


Boiling XstC.
Processing match 3426526483
Processing match 3423399440
Processing match 3417216544
Processing match 3417181431
Processing match 3412692791
Processing match 3406415169
Processing match 3405630092
Processing match 3403351345
Processing match 3396032748
Processing match 3362184678
Processing match 3362089600
Processing match 3361036594
Processing match 3360504618
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3353422199
Processing match 3326180456
Processing match 3319787769
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3318964387
Processing match 3317646669
Processing match 3311797119
Processing match 3311089379
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3311019454
Processing match 3310843323
Processing match 3309377536
Processing match 33075

Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3298515468
Processing match 3298427289
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3298421531
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3298315507
Processing match 3298290434
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3297334775
Processing match 3297349854
Processing match 3297245701
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3297210068
Processing match 3296628128
Processing match 3296101198
Processing match 3296074546
Processing match 3296062473
Processing match 3295608779
Processing match 3294675086
Processing match 3294691323
Matchup opponent identification error

Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:MID
Processing match 3322718972
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3322694471
Processing match 3322648968
Processing match 3322544113
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3322510420
Processing match 3322461232
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:TOP
Processing match 3322370529
Processing match 3322335939
Processing match 3322274295
Processing match 3322259838
Processing match 3321955020
Processing match 3321878016
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:MID
Processing match 3321781174
DONE
Boiling Antonah.
Processing match 3333572991
Processing match 3333546688
Processing match 3333457632
Processing match 3333

Processing match 3367283428
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3367215112
Processing match 3367150707
Processing match 3367004908
Processing match 3366942862
Processing match 3362673725
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3362069329
Processing match 3362111980
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3361984870
Processing match 3360564471
Processing match 3360078236
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3358604594
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3357578578
Processing match 3357630984
Processing match 3357505263
Matchup opponent identification error: ignoring results

...

Processing match 3369408081
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
DONE
Boiling Ass Hoe.
Processing match 3428300218
Processing match 3428190174
Processing match 3428057906
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3414636365
Processing match 3414602324
Processing match 3414526769
Processing match 3414551565
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3413964938
Processing match 3413989975
Processing match 3412739615
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3412764075
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3412750051
Matchup opponent identification error: ignoring results from match.
Unable to match role:

Processing match 3402904694
Processing match 3402909893
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3402807537
Processing match 3399789006
Processing match 3399709978
Processing match 3399686600
Processing match 3399644474
Processing match 3399619733
Processing match 3399587219
Processing match 3399583436
Processing match 3399621060
Processing match 3398430302
Processing match 3398127456
Processing match 3394327211
Processing match 3393959695
Processing match 3393846086
Processing match 3393816630
Processing match 3393772866
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3393756481
Processing match 3392512514
Processing match 3392496289
Processing match 3392490651
Processing match 3392407287
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3392213054
Processing matc

Processing match 2948683003
Processing match 2948675326
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 2948672737
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:TOP
DONE
Boiling Pakonab.
Processing match 3424548287
Processing match 3424156708
Processing match 3424110246
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3423024014
Processing match 3422984564
Processing match 3421988412
Processing match 3420419520
Processing match 3420239337
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3417810381
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3417516984
Processing match 3405176371
Processing match 3404329199
Processing match 3404125049
Processin

Processing match 3297847513
Processing match 3297823427
Processing match 3297758681
Processing match 3297782389
Processing match 3297732276
Processing match 3297675512
Processing match 3297619868
Processing match 3297561536
Processing match 3297545834
Processing match 3297521133
Processing match 3297486561
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3297462035
Processing match 3297425863
Processing match 3297381817
Processing match 3297356985
Processing match 3297351702
Processing match 3297266393
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3297272676
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3297218822
Processing match 3297213058
Processing match 3297188369
Processing match 3297175068
Processing match 3297129539
Processing match 3297105121
Processing m

Processing match 3303640840
Processing match 3303602183
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3303544835
Processing match 3303540052
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3303493959
Processing match 3303440044
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:MID
Processing match 3302668968
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3302653611
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3302598325
Processing match 3301661093
Processing match 3301557967
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3301573451
Processing match 3298897257
Processing match 3298110571
Matchup o

In [81]:
boil_league('SILVER_Rumble\'s Urfriders')

Boiling 190 players from SILVER_Rumble's Urfriders.


Boiling theonlyjman.
Processing match 3429415176
Processing match 3429298570
Processing match 3429312330
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:MID
Processing match 3419337382
Processing match 3417071225
Processing match 3416026568
Processing match 3416000470
Processing match 3414802031
Processing match 3412867300
Processing match 3412798633
Processing match 3406936147
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3406983232
Processing match 3406053494
Processing match 3403685603
Processing match 3403661553
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3403098230
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3403090212
Processing match 3401823310
Matchup opponent identification error: ignoring results from 

Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3133879533
Processing match 3133831476
Processing match 3133771432
Processing match 3133751632
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3133698945
Processing match 3133679487
Processing match 3133652985
Processing match 3133640973
Processing match 3133617998
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3133569708
Processing match 3133561332
Processing match 3133550552
Processing match 3133501571
Processing match 3133459414
Processing match 3133461422
Processing match 3133433637
Processing match 3133399710
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3133384620
Processing match 3133391443
Matchup opponent identification error: ignoring results 

Processing match 3072010793
Processing match 3071168582
Processing match 3071143405
Processing match 3070786455
Processing match 3070768255
Processing match 3070751210
Processing match 3070739907
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3070704753
Processing match 3070685810
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3070648314
Processing match 3070389432
Processing match 3070382252
Processing match 3070349272
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3070352685
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
DONE
Boiling Specia.
Processing match 3404416911
Processing match 3404410940
Processing match 3342765319
Processing match 3342779921
Matchup opponent identification error: ignoring results from m

Processing match 3319013691
Processing match 3318039525
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3318083987
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3318028419
Processing match 3317927302
Processing match 3317922431
Processing match 3317744915
Processing match 3317503384
Processing match 3317467451
Processing match 3316093444
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3316056469
Processing match 3316008430
Processing match 3315952617
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3315900810
Processing match 3315884680
Processing match 3315761387
Processing match 3315599188
Processing match 3315453937
Processing match 3315430109
Processing match 3315366874
Matchup opponent identifica

...

Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:MID
Processing match 3189370875
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3186112393
Processing match 3186081667
Processing match 3186090373
Processing match 3184905558
Processing match 3184943763
Processing match 3184902840
Processing match 3184940882
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3184859613
Processing match 3184847899
Processing match 3166816172
Processing match 3166783618
Processing match 3166746288
Processing match 3154906092
Processing match 3154894653
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:MID
Processing match 3154881037
Processing match 3154875476
Processing match 3154134710
Processing match 3154108476
Processing match 3153936278
Matchup oppo

Processing match 3243011158
Processing match 3242945406
Processing match 3239595427
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3237291641
Processing match 3237208879
Processing match 3236842306
Processing match 3236775141
Processing match 3236739695
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:TOP
Processing match 3235721938
Processing match 3235686453
Processing match 3235629276
Processing match 3235590445
Processing match 3231893823
Processing match 3231861445
Processing match 3227734765
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3227711803
Processing match 3227514847
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:BOTTOM
Processing match 3226381496
Processing match 3226316521
Matchup opponent identification error: ign

Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3422076762
Processing match 3419136625
Processing match 3419070976
Processing match 3419058935
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3419053091
Processing match 3418789653
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
Processing match 3417203672
Processing match 3416987086
Processing match 3416982526
Processing match 3416882812
Processing match 3416840152
Processing match 3414252920
Processing match 3414209524
Processing match 3414235556
Processing match 3413194020
Processing match 3408698731
Processing match 3408604427
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3408568526
Processing match 3406034001
Processing match 3406006003
Processing match 34044060

Processing match 3354646038
Processing match 3349308809
Processing match 3349264810
Processing match 3349302727
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3349189327
Processing match 3349194507
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_CARRY:BOTTOM
DONE
Boiling Downy Infused.
Processing match 3381411902
Processing match 3381326241
Processing match 3379940264
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3378127959
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:BOTTOM
Processing match 3378101307
Processing match 3376771566
Processing match 3376742396
Processing match 3376627635
Processing match 3375998601
Processing match 3375983773
Matchup opponent identification error: ignoring results from match.
Unable to match role: SOLO:MID
Processing match 3

Processing match 3115289593
Processing match 3115276981
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:TOP
Processing match 3107824382
Processing match 3107787535
Processing match 3107753419
Processing match 3107674959
Processing match 3107654953
Processing match 3107634395
Processing match 3107563027
Matchup opponent identification error: ignoring results from match.
Unable to match role: NONE:JUNGLE
Processing match 3107546578
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO_SUPPORT:NONE
Processing match 3107534002
Matchup opponent identification error: ignoring results from match.
Unable to match role: DUO:NONE
Processing match 3107503577
Processing match 3107473399
Processing match 3107432529
Processing match 3106616273
Processing match 3106433880
Processing match 3106415610
Processing match 3106388045
Processing match 3106371946
Processing match 3106361689
Processing match 310633618